Cài đặt BeautifulSoup

In [5]:
# %pip install beautifulsoup4
# %pip install requests


  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/57/ec/80c8d48ac8b1741d5b963797b7c0c869335619e13d4744ca2f67fc11c6fc/charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/c2/e7/a82b05cf63a603df6e68d59ae6a68bf5064484a0718ea5033660af4b54a9/idna-3.6-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/a2/73/a68704750a7679d0b6d3ad7aa8d4da8e14e151ae82e6fee774e6e0d05ec8/urllib3-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c962

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import thư viện

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import csv


Đối tượng thể hiện mỗi bài tuyển dụng

In [3]:
class Job:
    def __init__(self, title, link, company, salary, locations, level, yearOfExperience, contractType, sex, skills):
        self.title = title
        self.link = link
        self.company = company
        self.salary = salary
        self.locations = locations
        self.level = level
        self.yearOfExperience = yearOfExperience
        self.contractType = contractType
        self.sex = sex
        self.skills = skills
        

Hàm ghi một đối tượng công việc vào 1 hàng trong file .CSV

In [32]:
def write_to_csv(job):
    with open('jobs.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            # Write header row if file is empty
            if file.tell() == 0:
                writer.writerow(['Title', 'Link', 'Company', 'Salary', 'Locations', 'Level', 'Year of Experience', 'Contract Type', 'Sex', 'Skills'])
            writer.writerow([job.title, job.link, job.company, job.salary, job.locations, job.level, job.yearOfExperience, job.contractType, job.sex, job.skills])

Hàm lấy thông tin chi tiết của mỗi bài tuyển dụng

In [44]:
def get_jobs(item):
    print('********************* Start')
    try:
        title = item.find(class_='title').text.strip()
        print(title)
        link = item.find(class_='title').find('a')['href'].strip()
        print(link)
        company = item.find(class_='company').text.strip()
        print(company)
        salary = item.find(class_='title-salary').text.strip()
        print(salary)
        locations = item.find(class_='address').text.strip()
        print(locations)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        # Số lần thử lại tối đa
        max_retries = 5
        retry_delay = 1  # Thời gian chờ giữa các lần thử lại, đơn vị là giây

        retry_count = 0
        while retry_count < max_retries:
            response2 = requests.get(link, headers=headers)
            if response2.status_code == 429:  # Nếu nhận được mã trạng thái 429
                print("Too many requests. Retrying after waiting...")
                retry_count += 1
                time.sleep(retry_delay)  # Chờ trước khi thử lại
            else:
                # Xử lý kết quả ở đây nếu cần
                # Ví dụ: print(response.text)
                break
        else:
            print("Max retries exceeded. Request unsuccessful.")
        print(response2.status_code)
        if response2.status_code == 200:
            print('********************* Success')
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if link.startswith("https://www.topcv.vn/brand/"):
                infos = soup2.find_all(class_='box-info')
                generals = infos[0].find_all('span')
                level = generals[3].text.strip()
                print(level)
                yearOfExperience = generals[5].text.strip()
                print(yearOfExperience)
                contractType = generals[2].text.strip()
                print(contractType)
                sex = generals[4].text.strip()
                print(sex)
                skills = infos[2].find(class_='content-tab').text.strip()
                print(skills)
            else:
                generals = soup2.find(class_='job-detail__box--right job-detail__body-right--item job-detail__body-right--box-general').find_all(class_='box-general-group-info-value')
                level = generals[0].text.strip()
                print(level)
                yearOfExperience = generals[1].text.strip()
                print(yearOfExperience)
                contractType = generals[3].text.strip()
                print(contractType)
                sex = generals[4].text.strip()
                print(sex)
                skills = soup2.find_all(class_='job-description__item--content')[1].text.strip()
                print(skills)
        else:
            level = None
            yearOfExperience = None
            contractType = None
            sex = None
            skills = None
        job = Job(title, link, company, salary, locations, level, yearOfExperience, contractType, sex, skills)
        # Writing to CSV file
        write_to_csv(job)
    except:
        print('********************* Fail')
    print('********************* End')

Hiện tại khuyến khích crawl mỗi lần 1 trang sau 1 khoảng thời gian rồi crawl tiếp trang khác để tránh lỗi 429

Crawl 1 trang

In [ ]:
url = 'https://www.topcv.vn/viec-lam-it'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    # Sử dụng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    jobs = soup.find_all(class_='job-item-2')
    for job in jobs:
        get_jobs(job)   
else:
    print('Yêu cầu không thành công. Mã trạng thái:', response.status_code)

Crawl toàn bộ trang

In [45]:
url = 'https://www.topcv.vn/viec-lam-it'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    # Sử dụng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    num_pages = int(soup.find(class_='pagination').find_all('li')[-2].text)
    print(num_pages)
    jobs = soup.find_all(class_='job-item-2')
    for job in jobs:
        get_jobs(job)
    for i in range(2, num_pages + 1):
        url = 'https://www.topcv.vn/viec-lam-it?page=' + str(i)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            jobs = soup.find_all(class_='job-item-2')
            for job in jobs:
                get_jobs(job)
        else:
            print('Yêu cầu không thành công. Mã trạng thái:', response.status_code)              
else:
    print('Yêu cầu không thành công. Mã trạng thái:', response.status_code)

64
********************* Start
Trưởng Nhóm Triển Khai - Vận Hành Hệ Thống
https://www.topcv.vn/viec-lam/truong-nhom-trien-khai-van-hanh-he-thong/1269618.html?ta_source=ITJobs_LinkDetail
CHI NHÁNH CÔNG TY CỔ PHẦN PHẦN MỀM VIỄN THÔNG MIỀN NAM TẠI HÀ NỘI
15 - 20 triệu
Hà Nội
200
********************* Success
Trưởng nhóm
1 năm
Toàn thời gian
Không yêu cầu
- Tốt nghiệp từ Cao đẳng trở lên chuyên ngành CNTT hoặc các chuyên ngành liên quan;
- Có ít nhất 01 năm kinh nghiệm ở vị trí tương đương;
- Có kiến thức nền tảng với HDH Linux (Ubuntu, CentOS, ...), Vmware
- Có kiến thức nền tảng về Networking và hệ thống mạng LAN/WAN,Có kiến thức VoIP là lợi thế
- Có kỹ năng phân tích, tổng hợp và xử lý thông tin, dữ liệu CNTT;
- Có kỹ năng chẩn đoán và giải quyết vấn đề;
********************* End
********************* Start
Unity Developer - Lương Từ 12 - 30 Triệu - Được Chia Sẻ Doanh Thu Của Các Dự Án
https://www.topcv.vn/viec-lam/unity-developer-luong-tu-12-30-trieu-duoc-chia-se-doanh-thu-cua-cac-du-a